<a href="https://colab.research.google.com/github/anson1788/AiCardBackend/blob/main/opcg_turn_though.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama_index
!pip install llama-index-llms-groq
!pip install llama-index-llms-openai
!pip install llama-index-agent-openai
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core import PromptTemplate
import random
from pydantic import Field
from llama_index.agent.openai import OpenAIAgent

from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.llms import LLM
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.query_engine.custom import STR_OR_RESPONSE_TYPE
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata

class LLMQueryEngine(CustomQueryEngine):
    """
    This is a generic language model which can answer any question not handled \
by any other tool given. ALWAYS ask in complete questions.
    """

    system_prompt = """\
You are an advanced language model tasked with providing precise and comprehensive answers \
to user questions. Your responses should be as brief as possible, while including all necessary \
details to fully answer the question. Aim for clarity and completeness in every answer. \
Use clear and direct language, avoiding unnecessary words. Here are the key instructions to follow:

1. Understand the user's question thoroughly.
2. Answer with the minimum number of words needed to fully and accurately address the question.
3. Include all relevant details that are essential for a complete response.
4. Avoid extra information that doesn't directly answer the question.
5. Maintain a polite and professional tone.
6. Always answer in the language of the given question.

Always answer in normal text mode and only use structured format if they are part of your answer.
DO NOT prepend your answer with any label like 'assistat:' or 'answer:'.
The question will be in JSON format given by the USER below:
"""
    llm: LLM | None

    def __init__(self, llm: LLM):
        super().__init__()
        self.llm = llm

    def custom_query(self, query_str: str) -> STR_OR_RESPONSE_TYPE:
        """Run a custom query."""

        if isinstance(self.llm, LLM):
            chat_response = self.llm.chat(
                [
                    ChatMessage(role=MessageRole.SYSTEM, content=self.system_prompt),
                    ChatMessage(role=MessageRole.USER, content=query_str),
                ]
            )

            return str(chat_response.message.content)

        return ""




context ="""


"""
llm = Groq(model="llama3-8b-8192", api_key="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")

#agent = OpenAIAgent.from_tools([], llm=llm, verbose=True, context=context)
response = llm.complete("do u know one piece card game")
print(response)


INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


NameError: name 'agent' is not defined